In [1]:
import pandas as pd
import altair as alt
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [63]:
df = pd.read_csv('electricity.csv', parse_dates=['datetime'], dayfirst=True)
df = df.groupby([df["datetime"].dt.floor("1d")]).sum().reset_index()
df.columns = ['Date', 'nuclear', 'thermal', 'renewable', 'heat_power', 'hydro', 'pumped_hydro', 'consumption']

In [15]:
alt.Chart(df, title='Amount of energy produced by each type of production per day').transform_fold(
    ['nuclear',
     'thermal',
     'renewable',
     'heat_power',
     'hydro', 
     'pumped_hydro'],
    as_ = ['Production_type', 'amount']
).mark_area().encode(
    alt.Y('amount:Q'),
    alt.X('Date:T'),
    alt.Color('Production_type:N'),
    tooltip = [alt.Tooltip('Date:T'), alt.Tooltip('nuclear:Q'), alt.Tooltip('thermal:Q'), alt.Tooltip('renewable:Q'), alt.Tooltip('heat_power:Q'), alt.Tooltip('hydro:Q'), alt.Tooltip('pumped_hydro:Q')]
).properties(width = 800, height = 600)

alt.Chart(...)

Тут ми можемо побачити кількість єнергії, що виробляється з кожного типу джерел та яка різниця у, власне, кількості виробленої енергії по кожному типу джерел. Також тут видно які способи видобутку прогресують, а які занепадають.

In [64]:
df1 = df.groupby(df.Date.dt.to_period("M")).sum().reset_index()
df1['Date'] = pd.to_datetime(df1['Date'].apply(lambda x: x.strftime('%Y-%m')))

df1['production'] = df1['nuclear'] + df1['thermal'] + df1['renewable'] + df1['heat_power'] + df1['hydro'] + df1['pumped_hydro']
df1['saved'] = df1['production'] - df1['consumption']

df1['saved'] = df1['saved'].cumsum()

nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['Date'], empty='none')

line = alt.Chart(df1, title='Difference between produced and consumed energy for evey month and saved energy').transform_fold(
    ['production',
     'consumption',
     'saved'],
    as_ = ['Sums_for_the_whole_period', 'amount']
).mark_line().encode(
    alt.Y('amount:Q'),
    alt.X('Date:T'),
    alt.Color('Sums_for_the_whole_period:N'),
    tooltip = [alt.Tooltip('Date:T'), alt.Tooltip('consumption:Q'), alt.Tooltip('production:Q'), alt.Tooltip('difference:Q')]
)

selectors = alt.Chart(df1).mark_point().encode(
    x='Date:T',
    opacity=alt.value(0),
).add_selection(
    nearest
)
    
points = line.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)

text = line.mark_text(align='left', dx=5, dy=-5).encode(
    text=alt.condition(nearest, 'amount:Q', alt.value(' '))
)

rules = alt.Chart(df1).mark_rule(color='gray').encode(
    x='Date:T',
).transform_filter(
    nearest
)

alt.layer(
    line, selectors, points, rules, text
).properties(
    width=800, height=600
)

alt.LayerChart(...)

Тут можна порівняти енергію яка збирається і витрачається. Також видно скільки енергії збережено за весь термін (якщо зберігається)

In [125]:
df2 = df.groupby(df.Date.dt.to_period("M")).sum().reset_index()
df2['Date'] = pd.to_datetime(df2['Date'].apply(lambda x: x.strftime('%Y-%m')))
df2['Year'] = df2['Date'].dt.year
df2['Month'] = df2['Date'].dt.month_name()
df2 = pd.melt(df2, id_vars=['Year', 'Month'], 
              value_vars=['nuclear','thermal', 'renewable', 'heat_power', 'hydro', 'pumped_hydro'],
              var_name='Production_type', 
              value_name='amount')

alt.Chart(df2, title='Yearly and monthly view').mark_bar().encode(
    x='amount:Q',
    y='Month:N',
    color='Production_type:N',
    tooltip = [alt.Tooltip('amount:Q'), alt.Tooltip('Production_type:N')]

).properties(
    width=260, height=300
).facet(
    facet='Year:Q',
    columns=3
).resolve_scale(x = 'independent')

alt.FacetChart(...)

Тут можна побачити і порівняти кількість енергії, яка вироблялась в кожному місяці кожного року. Як бачимо, продуктивніть кожного типу вироблення енергії залежить від місяця.